# find inception score for mean images

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../")

%matplotlib notebook
import matplotlib.pylab as plt
# pylab.rcParams['figure.figsize'] = (10, 5)

In [2]:
import ld_gan
import numpy as np
from ld_gan import visualize
BATCH_SIZE = 256

# load data

In [3]:
X, Y = ld_gan.data_proc.data_loader.load_data(1, verbose=1, resize = 64)

100%|██████████| 102/102 [00:23<00:00,  7.17it/s]

load data from '/export/home/oblum/projects/ls_gan/data/flowers_102/jpg_128'


# compute inception score

### best

In [4]:
project, epoch = "xf_11111_s10l.py", 650
enc = ld_gan.utils.model_handler.load_model(project, epoch, "enc")
z_enc = ld_gan.utils.model_handler.apply_model(enc, X, batch_size=1000)
sampler = ld_gan.sample.nn_sampler(z_enc, X, Y, 1000, n_jobs = 10)

loaded model 'projects/xf_11111_s10l.py/model/e_0650.pth'


/net/hciserver03/storage/oblum/venvs/compvisgpu02/lib/python2.7/site-packages/torch/serialization.py:286: SourceChangeWarning: source code of class 'ld_gan.models.enc.Enc' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [5]:
_ = ld_gan.eval_gan.quick_score.i_score(project, epoch, X, sampler)

/net/hciserver03/storage/oblum/venvs/compvisgpu02/lib/python2.7/site-packages/torch/serialization.py:286: SourceChangeWarning: source code of class 'ld_gan.models.gen.Gen' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/net/hciserver03/storage/oblum/venvs/compvisgpu02/lib/python2.7/site-packages/torchvision/models/inception.py:65: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  m.weight.data.copy_(values)


loaded model 'projects/xf_11111_s10l.py/model/g_0650.pth'
compute inception score...
score_fake: 3.36714
compute inception score...
>= 50000 samples needed, but only 8189 provided
score_real: 3.56856
score     : 0.943557


### dcgan

In [6]:
project, epoch = "xf_11000_score.py", 650
sampler = ld_gan.sample.generate_rand_noise(X, Y, 1000, 256)

In [7]:
_ = ld_gan.eval_gan.quick_score.i_score(project, epoch, X, sampler)

loaded model 'projects/xf_11000_score.py/model/g_0650.pth'
compute inception score...
score_fake: 3.11624
compute inception score...
>= 50000 samples needed, but only 8189 provided
score_real: 3.56856
score     : 0.873249


# compute mode inception score

### best

In [8]:
project, epoch = "xf_11111_s10l.py", 650
enc = ld_gan.utils.model_handler.load_model(project, epoch, "enc")
z_enc = ld_gan.utils.model_handler.apply_model(enc, X, batch_size=1000)
sampler = ld_gan.sample.nn_sampler(z_enc, X, Y, 1000, n_jobs = 10)

loaded model 'projects/xf_11111_s10l.py/model/e_0650.pth'


In [10]:
_ = ld_gan.eval_gan.quick_score.im_score(project, epoch, X, sampler)

loaded model 'projects/xf_11111_s10l.py/model/g_0650.pth'
score_fake: 3.35762
score_real: 3.56856
score     : 0.940888


### dc gan

In [11]:
project, epoch = "xf_11000_score.py", 650
sampler = ld_gan.sample.generate_rand_noise(X, Y, 1000, 256)

In [12]:
_ = ld_gan.eval_gan.quick_score.im_score(project, epoch, X, sampler)

loaded model 'projects/xf_11000_score.py/model/g_0650.pth'
score_fake: 3.13267
score_real: 3.56856
score     : 0.877851
